# Постановка задачи

Рассмотреть различные метрики качества классификации, входящих в состав библиотеки scikit-learn. 
Для двух любых методов классификации из предыдущих работ и своего набора данных посчитать следующие метрики качества:
a. Точность классификации (Classification Accuracy)
b. Логарифм функции правдоподобия (Logarithmic Loss)
c. Область под кривой ошибок (Area Under ROC Curve)
d. Матрица неточностей (Confusion Matrix)
e. Отчет классификации (Classification Report)
Для более точных результатов использовать кросс-валидацию; cравнить применимость используемых классификаторов, основываясь на полученных метриках

# Исходные данные

•	Датасет: http://archive.ics.uci.edu/ml/datasets/seeds

•	Предметная область: семена пшениц

•	Задача: определить, к какому из 3х типов относится каждое семя (Kama, Rosa and Canadian)

•	Количество записей: 210

•	Количество атрибутов: 7

•	Атрибуты:

1. area A, 
2. perimeter P, 
3. compactness C = 4*pi*A/P^2, 
4. length of kernel, 
5. width of kernel, 
6. asymmetry coefficient 
7. length of kernel groove.

# Ход  работы


In [1]:
def split_data():
    ds = pandas.read_csv('seeds_dataset.txt', sep='\t', lineterminator='\n', header=None).values
    ds_attributes = ds[:, :-1]
    ds_class = ds[:, -1].astype(numpy.int64, copy=False)
    return ds_attributes, ds_class

def main():
    ds_attr, ds_class = split_data_cut()
    X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(ds_attr, ds_class, test_size=0.3, random_state=7)
    kFold=cross_validation.KFold(n=len(ds_attr),n_folds=10, random_state=7, shuffle=True)
    lda=LDA()
    gnb=GaussianNB()
    
    print("Accuracy of methods:")
    calc_accuracy(ds_attr, ds_class, kFold, lda, gnb)
    
    print("Logarithmic Loss Results:")
    calc_loss(ds_attr, ds_class, kFold, lda, gnb)
    
    print("Area Under ROC Curve Results: ")
    calc_curve(ds_attr, ds_class, kFold, lda, gnb)
    
    print("Confusion Matrixes:")
    calc_matrix(X_train, X_test, Y_train, Y_test, lda, gnb)

In [2]:
# Accuracy
def calc_accuracy(ds_attr, ds_class, kFold, lda, gnb):
    result = cross_val_score(lda, ds_attr, ds_class, cv=kFold, scoring='accuracy')
    print(" LDA:")
    print(" - mean: %0.5f" % result.mean())
    print(" - standart deviation: %0.5f" % result.std())

    result = cross_val_score(gnb, ds_attr, ds_class, cv=kFold, scoring='accuracy')
    print(" Gaussian:")
    print(" - mean: %0.5f" % result.mean())
    print(" - standart deviation: %0.5f" % result.std())

In [4]:
# Logarithmic Loss
def calc_loss(ds_attr, ds_class, kFold, lda, gnb):
    result = cross_validation.cross_val_score(lda, ds_attr, ds_class, cv=kFold, scoring='neg_log_loss')
    print(" LDA:")
    print(" - mean: %0.5f" % result.mean())
    print(" - standart deviation: %0.5f" % result.std())
    result = cross_validation.cross_val_score(gnb, ds_attr, ds_class, cv=kFold, scoring='neg_log_loss')
    print(" Gaussian:")
    print(" - mean: %0.5f" % result.mean())
    print(" - standart deviation: %0.5f" % result.std())

In [5]:
# Area Under ROC Curve
def calc_curve(ds_attr, ds_class, kFold, lda, gnb):
    result = cross_validation.cross_val_score(lda, ds_attr, ds_class, cv=kFold, scoring='roc_auc')
    print(" LDA:")
    print(" - mean: %0.5f" % result.mean())
    print(" - standart deviation: %0.5f" % result.std())
    result = cross_validation.cross_val_score(gnb, ds_attr, ds_class, cv=kFold, scoring='roc_auc')
    print(" Gaussian: %0.5f (%0.5f)" % (result.mean(), result.std() ))
    print(" - mean: %0.5f" % result.mean())
    print(" - standart deviation: %0.5f" % result.std())

In [6]:
# Confusion Matrix
def calc_matrix(X_train, X_test, Y_train, Y_test, lda, gnb):
    gnb.fit(X_train, Y_train)
    gnb_predicted = gnb.predict(X_test)
    gnb_matrix = confusion_matrix(Y_test, gnb_predicted)
    print(" - GaussianNB:")
    print(gnb_matrix)
    lda.fit(X_train,Y_train)
    lda_predicted=lda.predict(X_test)
    lda_matrix=confusion_matrix(Y_test,lda_predicted)
    print(" - LDA:")
    print(lda_matrix)
    
    # Classification Report
    print("Classification Reports:")
    lda_r=classification_report(Y_test,lda_predicted)
    print(' - LDA:')
    print(lda_r)
    gaus_r=classification_report(Y_test,gnb_predicted)
    print(" - GaussianNB:")
    print(gaus_r)

# Результаты

Accuracy of methods:
 LDA:
 - mean: 0.98571
 - standart deviation: 0.02857
 Gaussian:
 - mean: 0.95000
 - standart deviation: 0.05579
Logarithmic Loss Results:
 LDA:
 - mean: -0.02631
 - standart deviation: 0.04486
 Gaussian:
 - mean: -0.30403
 - standart deviation: 0.40455
Area Under ROC Curve Results:
 LDA:
 - mean: 1.00000
 - standart deviation: 0.00000
 Gaussian: 0.98541 (0.01618)
 - mean: 0.98541
 - standart deviation: 0.01618
Confusion Matrixes:
 - GaussianNB:
[[17  1  2]
 [ 1 22  0]
 [ 0  0 20]]
 - LDA:
[[19  0  1]
 [ 0 23  0]
 [ 0  0 20]]
Classification Reports:
 - LDA:
             precision    recall  f1-score   support

          1       1.00      0.95      0.97        20
          2       1.00      1.00      1.00        23
          3       0.95      1.00      0.98        20

avg / total       0.98      0.98      0.98        63

 - GaussianNB:
             precision    recall  f1-score   support

          1       0.94      0.85      0.89        20
          2       0.96      0.96      0.96        23
          3       0.91      1.00      0.95        20

avg / total       0.94      0.94      0.94        63

Согласно данным, полученным в ходе лабораторной работы, можно сделать вывод о высокой точности результатов, предоставляемых методами, и имеющих малую погрешность. Основываясь на матрице ошибок получено значение точности (precision), равное 0.98, а также полноты (recall), также равное 0.98. Эти значения подтверждают высокое качество получаемых результатов.